In [2]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tempfile
import pymupdf
import re
from dotenv import load_dotenv
import google.generativeai as genai
import time
import ast

In [108]:
testes = pd.read_excel('../docs/testes.xlsx')

acesso negado

In [6]:
def captura_num_processos(filtrados):
    copia_filtrados = filtrados.copy()
    sucessos = 0
    processos = []

    for i in range(copia_filtrados.shape[0]):
        link = copia_filtrados.iloc[i, 3]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""
        num_processo = "Não encontrado"

        if 'pdf' in content_type:
            # print(f"[PDF] Extraindo de: {link}")
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
            
        elif 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto)
        if match:
            num_processo = match.group(0)
            sucessos += 1

        processos.append(num_processo)
    
    print(f"Números de processos capturados: {sucessos}")

    copia_filtrados["num_processo"] = processos
    return copia_filtrados
    
filtrados_com_num = captura_num_processos(testes)
# filtrados_com_num.to_excel("processos_fitrados.xlsx")
filtrados_com_num

Números de processos capturados: 100


,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo
0,0800895-53.2022.8.10.0049,652049379,2643544408,http://jud-anexos.digesto.com.br/9d04a7ce4bf3c...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL PDF 636 64 KB,0800895-53.2022.8.10.0049
1,0800895-53.2022.8.10.0049,652049379,2643544419,http://jud-anexos.digesto.com.br/e512fdd683d21...,https://pje.tjma.jus.br/pje/Painel/painel_usua...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
2,0800895-53.2022.8.10.0049,652049379,2643544426,http://jud-anexos.digesto.com.br/80b47c8d75bae...,https://pje.tjma.jus.br/pje/Processo/ConsultaP...,LAUDO,2023-03-20,2025-02-25T02:50:29.170689,TJMA,LAUDO PERICIAL - LAUDO PERICIAL,0800895-53.2022.8.10.0049
3,5003677-41.2019.8.13.0521,612465137,590939495,http://jud-anexos.digesto.com.br/588fa00346ccf...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL PDF 1 437 88 KB,5003677-41.2019.8.13.0521
4,5003677-41.2019.8.13.0521,612465137,590939496,http://jud-anexos.digesto.com.br/f262789ad3d60...,https://pje.tjmg.jus.br/pje/Painel/painel_usua...,LAUDO,2020-09-06,2024-06-28T13:05:15.036135,TJMG,LAUDO PERICIAL - LAUDO PERICIAL,5003677-41.2019.8.13.0521
...,...,...,...,...,...,...,...,...,...,...,...
98,8001732-18.2024.8.05.0261,641758157,1542934097,http://jud-anexos.digesto.com.br/a1b0cc6140bfa...,https://pje.tjba.jus.br/pje/Processo/ConsultaP...,DOCUMENTO,2024-10-08,2024-11-08T21:17:59.156345,TJBA,DOCUMENTO DE COMPROVACAO - ALVARADELEVANTAMENTO,8001732-18.2024.8.05.0261
99,8001732-18.2024.8.05.0261,641758157,1542934101,http://jud-anexos.digesto.com.br/82219c92d2729...,https://pje.tjba.jus.br/pje/Processo/ConsultaP...,DOCUMENTO,2024-10-08,2024-11-08T21:17:59.156345,TJBA,DOCUMENTO DE COMPROVACAO - BANCOITAUCONSIGNADOSA,8001732-18.2024.8.05.0261
100,8001732-18.2024.8.05.0261,641758157,1542934102,http://jud-anexos.digesto.com.br/1be43c6232a87...,https://pje.tjba.jus.br/pje/Processo/ConsultaP...,DOCUMENTO,2024-10-08,2024-11-08T21:17:59.156345,TJBA,DOCUMENTO DE COMPROVACAO - 2SENTENCA8000534142...,8001732-18.2024.8.05.0261
101,8001732-18.2024.8.05.0261,641758157,1542934104,http://jud-anexos.digesto.com.br/483fc8cddfa24...,https://pje.tjba.jus.br/pje/Processo/ConsultaP...,DOCUMENTO,2024-10-08,2024-11-08T21:17:59.156345,TJBA,DOCUMENTO DE COMPROVACAO - 3LAUDOCUMPRIMENTOOBF,8001732-18.2024.8.05.0261


In [82]:
filtrados_com_num.to_excel("processos_fitrados.xlsx")


C:\Users\sergi\AppData\Local\Temp\ipykernel_41208\718792954.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  filtrados_com_num.to_excel("processos_fitrados.xlsx")


In [109]:
filtrados_com_num_excel = pd.read_excel(r"C:\Users\sergi\OneDrive\Área de Trabalho\INSPER\entidades\insper data\2025.1\aecom\repositorio\Projeto_InsperData_AECOM\testes_gemini\processos_filtrados.xlsx")

In [5]:
processos_nao_encontrados = filtrados_com_num.loc[filtrados_com_num['num_processo'] == "Não encontrado", :]

In [6]:
def identifica_acesso_negado(processos):
    copia_processos = processos.copy()
    acessos_negados = []

    for i in range(copia_processos.shape[0]):
        link = copia_processos.iloc[i, 3]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""

        if 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'Acesso negado', texto)
        if match:
            acessos_negados.append(i)
            continue
    
    return copia_processos.iloc[acessos_negados, :]

df_html_only = processos_nao_encontrados[processos_nao_encontrados['Download copia'].str.contains(r'\.html?$', case=False, na=False)]
acessos_negados = identifica_acesso_negado(df_html_only)

In [7]:
acessos_negados

,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo


Dentre os testes, nenhum acesso negado

# Utilização de Gemini 

Testar a validade de uso do Gemini a partir de uma amostra de testes da base de dados do jusbrasil.

In [ ]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

filtrados = testes
response = requests.get(filtrados.iloc[1,3])
response.encoding = 'utf-8'
content_type = response.headers.get('Content-Type', '')
texto_extraido = ""

if 'pdf' in content_type:
    # print(f"[PDF] Extraindo de: {link}")
    # Criando arquivo temporário para ler conteúdo do PDF 
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(response.content)
        tmp_path = tmp_file.name

    # Abrindo arquivo temporário e extraindo texto 
    with pymupdf.open(tmp_path) as doc:
        for page in doc:
            texto_extraido += page.get_text()
    
elif 'html' in content_type:
    # print(f"[HTML] Extraindo de: {link}")
    soup = BeautifulSoup(response.content, 'html.parser')
    texto_extraido = soup.get_text()

# Prompt
prompt = f"""Você está recebendo um texto bruto de um documento sobre
        algum processo judiciário. O seu interesse principal é descobrir se este texto trata de um caso relacionado à 
        valoração de danos ambientais.
        Assim, a partir das informações nele contidas, devolva APENAS UMA LISTA (NÃO DEVOLVA O TEXTO) 
        com as seguintes informações em ordem:
        Elemento 0 da lista: Texto fala de um processo em que houve algum tipo de dano ambiental (True ou False)
        Elemento 1 da lista: Texto menciona uma georreferência do local afetado pelo dano ambiental (devolva a string
        "Não encontrado" caso não exista)
        Elemento 2 da lista: A UF do local ou seja "Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará",
    "Distrito Federal", "Espírito Santo", "Goiás", "Maranhão",
    "Mato Grosso", "Mato Grosso do Sul", "Minas Gerais", "Pará",
    "Paraíba", "Paraná", "Pernambuco", "Piauí", "Rio de Janeiro",
    "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia", "Roraima",
    "Santa Catarina", "São Paulo", "Sergipe", "Tocantins" ou variações (devolva a string
        "Não encontrado" caso não exista)
        Elemento 3 da lista: O município ou cidade do local do processo (devolva a string
        "Não encontrado" caso não exista). 
        Elemento 4 da lista: O responsável pelo dano ambiental - aqui pode ser uma empresa ou pessoa física.
        (devolva a string "Não encontrado" caso não exista)
        Elemento 5 da lista: Categoria do elemento 4 - "Pessoa Física" ou "Pessoa Jurídica"
        (devolva a string "Não encontrado" caso não exista)
        Elemento 6 da lista: A data de ocorrência do impacto ambiental no formato DD/MM/AA
        (devolva a string "Não encontrado" caso não exista)
        Elemento 7 da lista: A extensão da área afetada pelo impacto ambiental
        (devolva a string "Não encontrado" caso não exista)
        Elemento 8 da lista: A unidade de medida do elemento 7
        (devolva a string "Não encontrado" caso não exista)
        Elemento 9 da lista: A metodologia de punição do responsável pelo impacto
        (devolva a string "Não encontrado" caso não exista)
        Elemento 10 da lista: O valor da multa que o responsável deve pagar pelas suas ações (em R$)
        (devolva a string "Não encontrado" caso não exista)
        Texto bruto: {texto_extraido}"""
generation_config_low_temp = genai.GenerationConfig(
    max_output_tokens=500,
    temperature=0.0,
)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
time.sleep(3)
resposta.text

"```\n[False, 'Não encontrado', 'Maranhão', 'Paço do Lumiar', 'BRK AMBIENTAL - MARANHÃO S.A', 'Pessoa Jurídica', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado']\n```\n"

## Teste do Gemini para algumas amostras

In [8]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [30]:
respostas = {}
filtrados = filtrados_com_num

for i in range(0,10):
    response = requests.get(filtrados.iloc[i,3])

    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    texto_extraido = ""

    if 'pdf' in content_type:
    # print(f"[PDF] Extraindo de: {link}")
    # Criando arquivo temporário para ler conteúdo do PDF 
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

    # Abrindo arquivo temporário e extraindo texto 
        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto_extraido += page.get_text()
    
    elif 'html' in content_type:
    # print(f"[HTML] Extraindo de: {link}")
        soup = BeautifulSoup(response.content, 'html.parser')
        texto_extraido = soup.get_text()

# Prompt
    prompt = f"""
    Tarefa: Você receberá um texto bruto que descreve um processo judicial. Seu objetivo é analisar esse texto e identificar se ele trata de um caso de dano ambiental e extrair 13 informações específicas, conforme as instruções abaixo.
    Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
    Retorno esperado (formato da resposta):
    A resposta deve ser apenas uma lista com 13 elementos, nesta ordem específica:

    [0] O texto trata de dano ambiental?
     Valores possíveis: True ou False

    [1] Trecho do texto que justifique a resposta acima.

    [2] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
     Se não houver, retorne: "Não encontrado georreferencia"

    [3] Unidade Federativa (UF) completa (ex: "Maranhão", "São Paulo" etc.)
     Se não houver, retorne: "Não encontrado UF"

    [4] Município ou cidade do local afetado.
     Se não houver, retorne: "Não encontrado cidade"

    [5] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
     Se não houver, retorne: "Não encontrado responsavel"

    [6] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
     Se não houver, retorne: "Não encontrado pessoa"

    [7] Data do impacto ambiental no formato: DD/MM/AA
     Se não houver, retorne: "Não encontrado data"

    [8] Extensão da área afetada (ex: "120 hectares", "15.000 m²", etc.)
     Se não houver, retorne: "Não encontrado área"

    [9] Unidade de medida da área (ex: "hectares", "m²", etc.)
     Se não houver, retorne: "Não encontrado medida"

    [10] Metodologia de punição aplicada (ex: auto de infração, processo administrativo, etc.)
     Se não houver, retorne: "Não encontrado método"

    [11] Valor da multa, compensação monetária ou indenização atribuída (em reais, formato: R$ 000.000,00)
     Se não houver, retorne: "Não encontrado valor"

    [12] Compensação não monetária atribuída  (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
     Se não houver, retorne: "Não encontrada compensação"

    IMPORTANTE: Não retorne o texto original, Não inclua explicações ou formatações extras, Apenas a lista com os 13 elementos na ordem acima, sem repetições. Não devolva o índice da lista, apenas os valores. Não inclua aspas ou colchetes na resposta. Se não encontrar algum elemento, retorne "Não encontrado" na posição correspondente da lista, apenas o valor da resposta.
    A resposta deve ser uma lista com EXATAMENTE 13 elementos (de índice 0 a 12), na ordem definida acima. Não retorne elementos extras. Não adicione vírgulas no final da lista. Não inclua o texto original, nem explicações, apenas a lista limpa.
    
    Texto bruto: {texto_extraido}"""
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=0.0,
)
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)
    lista = (resposta.text.split("\n"))
    lista.insert(0,filtrados.iloc[i,1])
    lista = [x for x in lista if x != '']
    match = re.search(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto_extraido)
    if match:
        num_processo = match.group(0)
        lista.append(num_processo)
    else:
        lista.append('Não encontrado')

    respostas[filtrados.iloc[i,2]] = lista
print(respostas)


{np.int64(2643544408): [np.int64(652049379), 'False', 'O texto trata de uma perícia em um processo judicial sobre cobranças de água consideradas altas por uma consumidora. Não há menção a danos ambientais.', 'Não encontrado georreferencia', 'Maranhão', 'Paço do Lumiar', 'BRK Ambiental Maranhão S.A', 'Pessoa Jurídica', 'Não encontrado data', '150', 'm²', 'Laudo pericial', 'Não encontrado valor', 'Não encontrada compensação', '0800895-53.2022.8.10.0049'], np.int64(2643544419): [np.int64(652049379), 'False', 'Não encontrado trecho', 'Não encontrado georreferencia', 'Maranhão', 'Não encontrado cidade', 'BRK AMBIENTAL - MARANHAO S.A', 'Pessoa Jurídica', 'Não encontrado data', 'Não encontrado área', 'Não encontrado medida', 'Não encontrado método', 'R$ 15.000,00', 'Não encontrada compensação', '0800895-53.2022.8.10.0049'], np.int64(2643544426): [np.int64(652049379), 'False', 'O texto trata de um processo judicial sobre cobranças de água consideradas abusivas pela consumidora,  baseado em per

In [31]:
#print((list(respostas.keys())))
respostas_teste = pd.DataFrame.from_dict(respostas, orient='index')

respostas_teste.columns = [
    "processoAnexoID",
    "processo_ambiental",
    "justificativa",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "área",
    "unidade",
    "metodologia_punição",
    "valor_multa",
    'compensacao_nao_monetaria',
    'num_processo']

In [ ]:
display(respostas_teste)

,processoAnexoID,processo_ambiental,justificativa,georreferência,UF,municipio,responsavel,tipo_responsavel,data,área,unidade,metodologia_punição,valor_multa,compensacao_nao_monetaria,num_processo
2643544408,652049379,False,O texto trata de uma perícia em um processo ju...,Não encontrado georreferencia,Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado data,150,m²,Laudo pericial,Não encontrado valor,Não encontrada compensação,0800895-53.2022.8.10.0049
2643544419,652049379,False,Não encontrado trecho,Não encontrado georreferencia,Maranhão,Não encontrado cidade,BRK AMBIENTAL - MARANHAO S.A,Pessoa Jurídica,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,"R$ 15.000,00",Não encontrada compensação,0800895-53.2022.8.10.0049
2643544426,652049379,False,O texto trata de um processo judicial sobre co...,Não encontrado georreferencia,Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado data,150,m²,Laudo pericial,Não encontrado valor,Não encontrada compensação,0800895-53.2022.8.10.0049
590939495,612465137,True,Foi realizada supressão de vegetação rasteira ...,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,06/08/2020,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Plano de recuperação para conter a erosão; cer...,5003677-41.2019.8.13.0521
590939496,612465137,True,Ação Civil Pública Cível referente a Área de P...,Não encontrado georreferencia,Não encontrado UF,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Não encontrada compensação,5003677-41.2019.8.13.0521
590939497,612465137,True,Foi realizada supressão de vegetação rasteira ...,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,06/08/2020,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,"Plano de recuperação para conter a erosão, cer...",5003677-41.2019.8.13.0521
1186042345,582564055,True,O processo trata de ação de reparação de danos...,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Vale S.A.,Pessoa Jurídica,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,"R$ 1.000.000,00",Não encontrada compensação,0000527-75.2011.8.02.0020
2589045924,690025576,True,O presente Laudo Técnico Pericial tem como obj...,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,CARMENSE COMERCIAL LTDA,Pessoa Jurídica,04/05/2023,Não encontrado área,Não encontrado medida,Processo administrativo,"R$ 3.807.896,40",desativação e a desmobilização completa do par...,0011297-52.2011.8.13.0140
2589045929,690025576,True,Mancha de óleo sobre o solo – Risco de contami...,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Carmense Comercial Ltda,Pessoa Jurídica,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Não encontrada compensação,0011297-52.2011.8.13.0140
2589045772,690025576,True,ÁREA DE ATUAÇÃO: Meio Ambiente,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,Carmense Comercial Ltda,Pessoa Jurídica,04/05/2023,Não encontrado área,Não encontrado medida,Processo Judicial Eletrônico,Não encontrado valor,Doação de 50ha na estação ecológica Mata do Ce...,0011297-52.2011.8.13.0140


## Processos 1 a 50

In [33]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

filtrados = testes
respostas_amostra1 = {}

In [34]:
def analise(i,respostas):
    response = requests.get(filtrados.iloc[i,3])

    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    texto_extraido = ""

    if 'pdf' in content_type:
    # print(f"[PDF] Extraindo de: {link}")
    # Criando arquivo temporário para ler conteúdo do PDF 
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

    # Abrindo arquivo temporário e extraindo texto 
        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto_extraido += page.get_text()
    
    elif 'html' in content_type:
    # print(f"[HTML] Extraindo de: {link}")
        soup = BeautifulSoup(response.content, 'html.parser')
        texto_extraido = soup.get_text()

# Prompt
    prompt = f"""
    Tarefa: Você receberá um texto bruto que descreve um processo judicial. Seu objetivo é analisar esse texto e identificar se ele trata de um caso de dano ambiental e extrair 13 informações específicas, conforme as instruções abaixo.
    Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
    Retorno esperado (formato da resposta):
    A resposta deve ser apenas uma lista com 13 elementos, nesta ordem específica:

    [0] O texto trata de dano ambiental?
     Valores possíveis: True ou False

    [1] Trecho do texto que justifique a resposta acima.

    [2] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
     Se não houver, retorne: "Não encontrado georreferencia"

    [3] Unidade Federativa (UF) completa (ex: "Maranhão", "São Paulo" etc.)
     Se não houver, retorne: "Não encontrado UF"

    [4] Município ou cidade do local afetado.
     Se não houver, retorne: "Não encontrado cidade"

    [5] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
     Se não houver, retorne: "Não encontrado responsavel"

    [6] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
     Se não houver, retorne: "Não encontrado pessoa"

    [7] Data do impacto ambiental no formato: DD/MM/AA
     Se não houver, retorne: "Não encontrado data"

    [8] Extensão da área afetada (ex: "120 hectares", "15.000 m²", etc.)
     Se não houver, retorne: "Não encontrado área"

    [9] Unidade de medida da área (ex: "hectares", "m²", etc.)
     Se não houver, retorne: "Não encontrado medida"

    [10] Metodologia de punição aplicada (ex: auto de infração, processo administrativo, etc.)
     Se não houver, retorne: "Não encontrado método"

    [11] Valor da multa, compensação monetária ou indenização atribuída (em reais, formato: R$ 000.000,00)
     Se não houver, retorne: "Não encontrado valor"

    [12] Compensação não monetária atribuída  (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
     Se não houver, retorne: "Não encontrada compensação"

    IMPORTANTE: Não retorne o texto original, Não inclua explicações ou formatações extras, Apenas a lista com os 13 elementos na ordem acima, sem repetições. Não devolva o índice da lista, apenas os valores. Não inclua aspas ou colchetes na resposta. Se não encontrar algum elemento, retorne "Não encontrado" na posição correspondente da lista, apenas o valor da resposta.
    A resposta deve ser uma lista com EXATAMENTE 13 elementos (de índice 0 a 12), na ordem definida acima. Não retorne elementos extras. Não adicione vírgulas no final da lista. Não inclua o texto original, nem explicações, apenas a lista limpa.
    
    Texto bruto: {texto_extraido}"""
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=0.0,
)
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)
    lista = (resposta.text.split("\n"))
    lista.insert(0,filtrados.iloc[i,1])
    lista = [x for x in lista if x != '']
    match = re.search(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto_extraido)
    if match:
        num_processo = match.group(0)
        lista.append(num_processo)
    else:
        lista.append('Não encontrado')

    respostas[filtrados.iloc[i,2]] = lista

In [36]:
for i in range(0,50):
    analise(i,respostas_amostra1)
    time.sleep(3)
    # print(respostas_amostra1)

In [37]:
#print((list(respostas.keys())))
respostas_teste_amostra1 = pd.DataFrame.from_dict(respostas_amostra1, orient='index')

respostas_teste_amostra1.columns = [
    "processoAnexoID",
    "processo_ambiental",
    "justificativa",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "área",
    "unidade",
    "metodologia_punição",
    "valor_multa",
    'compensacao_nao_monetaria',
    'num_processo']

In [38]:
respostas_teste_amostra1

,processoAnexoID,processo_ambiental,justificativa,georreferência,UF,municipio,responsavel,tipo_responsavel,data,área,unidade,metodologia_punição,valor_multa,compensacao_nao_monetaria,num_processo
2643544408,652049379,False,O texto trata de uma perícia em um processo ju...,Não encontrado georreferencia,Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado data,150,m²,Laudo técnico pericial,Não encontrado valor,Não encontrada compensação,0800895-53.2022.8.10.0049
2643544419,652049379,False,Não encontrado trecho,Não encontrado georreferencia,Maranhão,Não encontrado cidade,BRK AMBIENTAL - MARANHÃO S.A,Pessoa Jurídica,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,"R$ 15.000,00",Não encontrada compensação,0800895-53.2022.8.10.0049
2643544426,652049379,False,O texto trata de um processo judicial sobre co...,Não encontrado georreferencia,Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado data,150,m²,Laudo pericial,Não encontrado valor,Não encontrada compensação,0800895-53.2022.8.10.0049
590939495,612465137,True,Foi realizada supressão de vegetação rasteira ...,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,06/08/2020,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Plano de recuperação para conter a erosão; cer...,5003677-41.2019.8.13.0521
590939496,612465137,True,Ação Civil Pública Cível referente a Área de P...,Não encontrado georreferencia,Não encontrado UF,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Não encontrada compensação,5003677-41.2019.8.13.0521
590939497,612465137,True,Foi realizada supressão de vegetação rasteira ...,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,06/08/2020,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,"Plano de recuperação para conter a erosão, cer...",5003677-41.2019.8.13.0521
1186042345,582564055,True,O processo trata de ação de reparação de danos...,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Não encontrado responsavel,Não encontrado pessoa,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Não encontrada compensação,0000527-75.2011.8.02.0020
2589045924,690025576,True,O presente Laudo Técnico Pericial tem como obj...,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,CARMENSE COMERCIAL LTDA,Pessoa Jurídica,04/05/2023,Não encontrado área,Não encontrado medida,Processo administrativo,"R$ 3.807.896,40",desativação e a desmobilização completa do par...,0011297-52.2011.8.13.0140
2589045929,690025576,True,Mancha de óleo sobre o solo – Risco de contami...,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Carmense Comercial Ltda,Pessoa Jurídica,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Não encontrada compensação,0011297-52.2011.8.13.0140
2589045772,690025576,True,ÁREA DE ATUAÇÃO: Meio Ambiente,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,Carmense Comercial Ltda,Pessoa Jurídica,04/05/2023,Não encontrado área,Não encontrado medida,Processo Judicial Eletrônico,Não encontrado valor,Doação de 50ha na estação ecológica Mata do Ce...,0011297-52.2011.8.13.0140


In [39]:
respostas_teste_amostra1.to_excel("testes_amostra1.xlsx")

## Processos 51 a 100

In [40]:
respostas_amostra2 = {}
for i in range(50,103):
    analise(i,respostas_amostra2)
    time.sleep(2)

In [41]:
respostas_teste_amostra2 = pd.DataFrame.from_dict(respostas_amostra2, orient='index')

respostas_teste_amostra2.columns = [
    "processoAnexoID",
    "processo_ambiental",
    "justificativa",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "área",
    "unidade",
    "metodologia_punição",
    "valor_multa",
    'compensacao_nao_monetaria',
    'num_processo']

In [42]:
respostas_teste_amostra2.to_excel("testes_amostra2.xlsx")

# Juntar procesos

In [21]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

In [110]:
filtrados_com_num = filtrados_com_num_excel
processos_unicos = list(filtrados_com_num['num_processo'].unique())
display(filtrados_com_num.loc[filtrados_com_num['num_processo']=='Não encontrado'])
processos_unicos.remove('Não encontrado')

,Unnamed: 0,processoID,processoAnexoID,Download copia,Tribunal,num_processo
4131,169,655333385,1942855138,http://jud-anexos.digesto.com.br/30f8d4e93b94a...,TJBA,Não encontrado
4132,173,655333385,1942855153,http://jud-anexos.digesto.com.br/f2d578e94d700...,TJBA,Não encontrado
4133,194,655333385,1942855177,http://jud-anexos.digesto.com.br/ac2380dd6d952...,TJBA,Não encontrado
4134,195,655333385,1942855178,http://jud-anexos.digesto.com.br/042db8c607606...,TJBA,Não encontrado
4135,196,655333385,1942855179,http://jud-anexos.digesto.com.br/a34562b835d2a...,TJBA,Não encontrado
...,...,...,...,...,...,...
4293,13855,494410810,257604195,http://jud-anexos.digesto.com.br/550b4f243c10a...,TRF1,Não encontrado
4294,13856,494410810,257604196,http://jud-anexos.digesto.com.br/ec272e1878687...,TRF1,Não encontrado
4295,13857,494410810,257604197,http://jud-anexos.digesto.com.br/b645d0f74e2e9...,TRF1,Não encontrado
4296,13858,494410810,257604198,http://jud-anexos.digesto.com.br/124da1a65a0e3...,TRF1,Não encontrado


In [126]:
def analise(texto_extraido,respostas,numero_processo,processoAnexoID,link):

    prompt = f"""
    Tarefa: Você receberá um texto bruto que descreve um processo judicial. Seu objetivo é analisar esse texto e identificar se ele trata de um caso de dano ambiental e extrair 13 informações específicas, conforme as instruções abaixo.
    Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
    Retorno esperado (formato da resposta):
    A resposta deve ser apenas uma lista com 13 elementos, nesta ordem específica:

    [0] O texto trata de dano ambiental?
     Valores possíveis: True ou False

    [1] Trecho do texto que justifique a resposta acima.

    [2] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
     Se não houver, retorne: "Não encontrado georreferencia"

    [3] Unidade Federativa (UF) completa (ex: "Maranhão", "São Paulo" etc.)
     Se não houver, retorne: "Não encontrado UF"

    [4] Município ou cidade do local afetado.
     Se não houver, retorne: "Não encontrado cidade"

    [5] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
     Se não houver, retorne: "Não encontrado responsavel"

    [6] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
     Se não houver, retorne: "Não encontrado pessoa"

    [7] Data do impacto ambiental no formato: DD/MM/AA
     Se não houver, retorne: "Não encontrado data"

    [8] Extensão da área afetada (ex: "120 hectares", "15.000 m²", etc.)
     Se não houver, retorne: "Não encontrado área"

    [9] Unidade de medida da área (ex: "hectares", "m²", etc.)
     Se não houver, retorne: "Não encontrado medida"

    [10] Metodologia de punição aplicada (ex: auto de infração, processo administrativo, etc.)
     Se não houver, retorne: "Não encontrado método"

    [11] Valor da multa, compensação monetária ou indenização atribuída (em reais, formato: R$ 000.000,00)
     Se não houver, retorne: "Não encontrado valor"

    [12] Compensação não monetária atribuída  (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
     Se não houver, retorne: "Não encontrada compensação"

    IMPORTANTE: Não retorne o texto original, Não inclua explicações ou formatações extras, Apenas a lista com os 13 elementos na ordem acima, sem repetições. Não devolva o índice da lista, apenas os valores. Não inclua aspas ou colchetes na resposta. Se não encontrar algum elemento, retorne "Não encontrado" na posição correspondente da lista, apenas o valor da resposta.
    A resposta deve ser uma lista com EXATAMENTE 13 elementos (de índice 0 a 12), na ordem definida acima. Não retorne elementos extras. Não adicione vírgulas no final da lista. Não inclua o texto original, nem explicações, apenas a lista limpa.
    
    Texto bruto: {texto_extraido}"""
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=0.0,
)
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)
    lista = (resposta.text.split("\n"))
    lista.insert(0,numero_processo)
    lista = [x for x in lista if x != '']
    lista.append(processoAnexoID)
    lista.append(link)
    
    respostas[numero_processo] = lista
    return respostas
    

In [127]:
processos_unicos_teste = processos_unicos[0:3]
processos_unicos_teste

['0000010-75.2024.8.02.0356',
 '0000135-52.2021.8.16.0067',
 '0000156-20.2023.8.02.0076']

In [ ]:
respostas = {}

for processo in processos_unicos_teste:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    analise(texto,respostas,processo,processoAnexoID,link)



In [131]:
respostas_teste2 = pd.DataFrame.from_dict(respostas, orient='index')

respostas_teste2.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "área",
    "unidade",
    "metodologia_punição",
    "valor_multa",
    'compensacao_nao_monetaria',
    'processoAnexoID',
    'link'
    ]

In [134]:
for i in range(0,3):
    print(respostas_teste2.iloc[i,15])

http://jud-anexos.digesto.com.br/c1541200bd444765f2ceab1bcb73e591.pdf
http://jud-anexos.digesto.com.br/a5625273387d03675117df077783ffb2.pdf
http://jud-anexos.digesto.com.br/25afbf7fded56e9b963e05b9f2b6bb1b.pdf


In [135]:
respostas_teste2

,processo,processo_ambiental,justificativa,georreferência,UF,municipio,responsavel,tipo_responsavel,data,área,unidade,metodologia_punição,valor_multa,compensacao_nao_monetaria,processoAnexoID,link
0000010-75.2024.8.02.0356,0000010-75.2024.8.02.0356,False,O texto descreve uma ação judicial por danos m...,Não encontrado georreferencia,Alagoas,União dos Palmares,Verde Ambiental Alagoas S.a,Pessoa Jurídica,10/2023,Não encontrado área,Não encontrado medida,Procedimento do Juizado Especial Cível,"R$ 700,00",Não encontrada compensação,1676545211,http://jud-anexos.digesto.com.br/c1541200bd444...
0000135-52.2021.8.16.0067,0000135-52.2021.8.16.0067,True,Trata-se de Ação Civil Pública ajuizada pelo M...,Não encontrado georreferencia,Paraná,Cerro Azul,Nataniel do Nascimento,Pessoa Física,23/07/2019,Não encontrado área,Não encontrado medida,Ação Civil Pública,"R$ 16.459,56",Não encontrada compensação,2908924070,http://jud-anexos.digesto.com.br/a5625273387d0...
0000156-20.2023.8.02.0076,0000156-20.2023.8.02.0076,False,O texto trata de uma ação de indenização por d...,Não encontrado georreferencia,Alagoas,Maceió,Apple Computer Brasil Ltda,Pessoa Jurídica,15/08/2023,Não encontrado área,Não encontrado medida,Processo judicial,"R$ 3.000,00",Não encontrada compensação,1419499656,http://jud-anexos.digesto.com.br/25afbf7fded56...


## é ou não é dano ambiental?

In [166]:
def analise_dano_ambiental(texto_extraido,respostas,numero_processo,processoAnexoID,link):

    prompt = f"""
    Tarefa: Você receberá um texto bruto que descreve um processo judicial. Seu objetivo é analisar esse texto e identificar se ele trata de um caso de dano ambiental e extrair 13 informações específicas, conforme as instruções abaixo.
    Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
    Retorno esperado (formato da resposta):
    A resposta deve ser apenas uma lista com 13 elementos, nesta ordem específica:

    [0] O texto trata de dano ambiental?
     Valores possíveis: True ou False

    [1] Trecho do texto que justifique a resposta acima.

    IMPORTANTE: Não inclua explicações ou formatações extras, Apenas a lista com os 2 elementos na ordem acima, sem repetições. Não devolva o índice da lista, apenas os valores. Não inclua aspas ou colchetes na resposta. Se não encontrar algum elemento, retorne "Não encontrado" na posição correspondente da lista, apenas o valor da resposta.
    A resposta deve ser uma lista com EXATAMENTE 2 elementos (de índice 0 a 1), na ordem definida acima. Não retorne elementos extras. Não adicione vírgulas no final da lista. Não inclua o texto original, nem explicações, apenas a lista limpa.
    
    Texto bruto: {texto_extraido}"""
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=0.0,
)
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)
    lista = (resposta.text.split("\n"))
    lista.insert(0,numero_processo)
    lista = [x for x in lista if x != '']
    lista.append(processoAnexoID)
    lista.append(link)
    
    respostas[numero_processo] = lista
    #print(respostas)
    return respostas
    

In [167]:
processos_unicos_50 = processos_unicos[0:50]

In [168]:
respostas_dano_ambiental_50 = {}

for processo in processos_unicos_50:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    analise_dano_ambiental(texto,respostas_dano_ambiental_50,processo,processoAnexoID,link)



In [169]:
respostas_danoambiental_50 = pd.DataFrame.from_dict(respostas_dano_ambiental_50, orient='index')

respostas_danoambiental_50.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

In [170]:
respostas_danoambiental_50

,processo,processo_ambiental,justificativa,processoAnexoID,link de referencia
0000010-75.2024.8.02.0356,0000010-75.2024.8.02.0356,False,O texto descreve uma ação judicial por danos m...,1676545211,http://jud-anexos.digesto.com.br/c1541200bd444...
0000135-52.2021.8.16.0067,0000135-52.2021.8.16.0067,True,Trata-se de Ação Civil Pública ajuizada pelo M...,2908924070,http://jud-anexos.digesto.com.br/a5625273387d0...
0000156-20.2023.8.02.0076,0000156-20.2023.8.02.0076,False,O texto trata de uma ação de indenização por d...,1419499656,http://jud-anexos.digesto.com.br/25afbf7fded56...
0000272-92.1992.8.26.0157,0000272-92.1992.8.26.0157,True,Ação Civil Pública - Meio Ambiente,657568057,http://jud-anexos.digesto.com.br/4f696a2b03b85...
0000428-83.2019.8.16.0134,0000428-83.2019.8.16.0134,True,Assunto Principal: Dano Ambiental,1088560478,http://jud-anexos.digesto.com.br/05d31619ede92...
0000527-75.2011.8.02.0020,0000527-75.2011.8.02.0020,True,"Na petição inicial, dentre seus pedidos, os au...",1186043580,http://jud-anexos.digesto.com.br/7487b4175300b...
0000627-51.2015.8.16.0068,0000627-51.2015.8.16.0068,False,Assunto Principal: Indenização por Dano Moral,1397328389,http://jud-anexos.digesto.com.br/6b0828e58be5a...
0000640-15.2017.8.11.0102,0000640-15.2017.8.11.0102,True,O Ministério Público do Estado de Mato Grosso ...,1299854564,http://jud-anexos.digesto.com.br/40b55644f0bfc...
0001293-11.2021.8.16.0046,0001293-11.2021.8.16.0046,True,Trata-se de “ação civil pública por dano ambie...,1462784031,http://jud-anexos.digesto.com.br/a7e7d6002b57d...
0001700-76.2018.8.17.3220,0001700-76.2018.8.17.3220,False,O texto trata de ação declaratória de inexistê...,1484217656,http://jud-anexos.digesto.com.br/6a3d66555a358...


In [171]:
respostas_danoambiental_50.to_excel("respostas_danoambiental_50.xlsx")


C:\Users\sergi\AppData\Local\Temp\ipykernel_41208\2662321025.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_danoambiental_50.to_excel("respostas_danoambiental_50.xlsx")


In [172]:
processos_unicos_51_100 = processos_unicos[51:100]

respostas_dano_ambiental_51_100 = {}

for processo in processos_unicos_51_100:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    analise_dano_ambiental(texto,respostas_dano_ambiental_51_100,processo,processoAnexoID,link)

respostas_danoambiental_51_100 = pd.DataFrame.from_dict(respostas_dano_ambiental_51_100, orient='index')

respostas_danoambiental_51_100.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

respostas_danoambiental_51_100.to_excel("respostas_danoambiental_51_100.xlsx")

C:\Users\sergi\AppData\Local\Temp\ipykernel_41208\3087944915.py:42: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_danoambiental_51_100.to_excel("respostas_danoambiental_51_100.xlsx")


In [ ]:
respostas_danoambiental_51_100

In [176]:
processos_unicos_100_120 = processos_unicos[100:120]

respostas_dano_ambiental_100_120 = {}

for processo in processos_unicos_100_120:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    analise_dano_ambiental(texto,respostas_dano_ambiental_100_120,processo,processoAnexoID,link)

respostas_dano_ambiental_100_120 = pd.DataFrame.from_dict(respostas_dano_ambiental_100_120, orient='index')

respostas_dano_ambiental_100_120.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

respostas_dano_ambiental_100_120.to_excel("respostas_danoambiental_100_150.xlsx")

C:\Users\sergi\AppData\Local\Temp\ipykernel_41208\2404524492.py:42: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_dano_ambiental_100_120.to_excel("respostas_danoambiental_100_150.xlsx")


In [188]:
processos_unicos_120_130 = processos_unicos[120:130]

respostas_dano_ambiental_120_130 = {}

for processo in processos_unicos_120_130:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    analise_dano_ambiental(texto,respostas_dano_ambiental_120_130,processo,processoAnexoID,link)

respostas_dano_ambiental_120_130 = pd.DataFrame.from_dict(respostas_dano_ambiental_120_130, orient='index')

respostas_dano_ambiental_120_130.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

respostas_dano_ambiental_120_130.to_excel("respostas_dano_ambiental_120_130.xlsx")

C:\Users\sergi\AppData\Local\Temp\ipykernel_41208\1527285759.py:42: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_dano_ambiental_120_130.to_excel("respostas_dano_ambiental_120_130.xlsx")


In [ ]:
processos_unicos_130_140 = processos_unicos[130:140]

respostas_dano_ambiental_130_140 = {}

for processo in processos_unicos_130_140:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    print(processoAnexoID)
    analise_dano_ambiental(texto,respostas_dano_ambiental_130_140,processo,processoAnexoID,link)

respostas_dano_ambiental_130_140 = pd.DataFrame.from_dict(respostas_dano_ambiental_130_140, orient='index')

respostas_dano_ambiental_130_140.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

respostas_dano_ambiental_130_140.to_excel("respostas_dano_ambiental_130_140.xlsx")

2063414504
2711038736
113630275
116536345
681474840
113722464
1302699435
2761350629


InvalidArgument: 400 The input token count (1047951) exceeds the maximum number of tokens allowed (1000000).

In [ ]:
respostas_dano_ambiental_130_140 = pd.DataFrame.from_dict(respostas_dano_ambiental_130_140, orient='index')

respostas_dano_ambiental_130_140.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

respostas_dano_ambiental_130_140.to_excel("respostas_dano_ambiental_130_138.xlsx")

#os que ja foram:
# 2063414504
# 2711038736
# 113630275
# 116536345
# 681474840
# 113722464
# 1302699435
# 2761350629

C:\Users\sergi\AppData\Local\Temp\ipykernel_41208\3486571371.py:11: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_dano_ambiental_130_140.to_excel("respostas_dano_ambiental_130_138.xlsx")


In [193]:
processos_unicos_140_150 = processos_unicos[140:150]

respostas_dano_ambiental_140_150 = {}

for processo in processos_unicos_140_150:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    analise_dano_ambiental(texto,respostas_dano_ambiental_140_150,processo,processoAnexoID,link)

respostas_dano_ambiental_140_150 = pd.DataFrame.from_dict(respostas_dano_ambiental_140_150, orient='index')

respostas_dano_ambiental_140_150.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

respostas_dano_ambiental_140_150.to_excel("respostas_dano_ambiental_140_150.xlsx")

C:\Users\sergi\AppData\Local\Temp\ipykernel_41208\457592980.py:42: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_dano_ambiental_140_150.to_excel("respostas_dano_ambiental_140_150.xlsx")


In [202]:
processos_unicos_150_170 = processos_unicos[150:170]

respostas_dano_ambiental_150_170 = {}

for processo in processos_unicos_150_170:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    analise_dano_ambiental(texto,respostas_dano_ambiental_150_170,processo,processoAnexoID,link)

respostas_dano_ambiental_150_170 = pd.DataFrame.from_dict(respostas_dano_ambiental_150_170, orient='index')

respostas_dano_ambiental_150_170.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

respostas_dano_ambiental_150_170.to_excel("respostas_dano_ambiental_150_170.xlsx")

C:\Users\sergi\AppData\Local\Temp\ipykernel_41208\243550075.py:42: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_dano_ambiental_150_170.to_excel("respostas_dano_ambiental_150_170.xlsx")


In [209]:
processos_unicos_170_200 = processos_unicos[170:200]

respostas_dano_ambiental_170_200 = {}

for processo in processos_unicos_170_200:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    analise_dano_ambiental(texto,respostas_dano_ambiental_170_200,processo,processoAnexoID,link)

respostas_dano_ambiental_170_200 = pd.DataFrame.from_dict(respostas_dano_ambiental_170_200, orient='index')

respostas_dano_ambiental_170_200.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

respostas_dano_ambiental_170_200.to_excel("respostas_dano_ambiental_170_200.xlsx")

C:\Users\sergi\AppData\Local\Temp\ipykernel_41208\3429139937.py:42: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_dano_ambiental_170_200.to_excel("respostas_dano_ambiental_170_200.xlsx")


In [ ]:
processos_unicos_200_228 = processos_unicos[200:len(processos_unicos)]

respostas_dano_ambiental_200_228 = {}

for processo in processos_unicos_200_228:
    linhas_correspondentes = filtrados_com_num.loc[filtrados_com_num['num_processo'] == processo]
    texto = ''
   
    for index, row in linhas_correspondentes.iterrows():
        link = row['Download copia']
        processoAnexoID = row['processoAnexoID']
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        
        if 'pdf' in content_type:
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
        
        elif 'html' in content_type:
            soup = BeautifulSoup(response.content, 'html.parser')
            texto += soup.get_text()
    analise_dano_ambiental(texto,respostas_dano_ambiental_200_228,processo,processoAnexoID,link)

respostas_dano_ambiental_200_228 = pd.DataFrame.from_dict(respostas_dano_ambiental_200_228, orient='index')

respostas_dano_ambiental_200_228.columns = [
    "processo",
    "processo_ambiental",
    "justificativa",
    'processoAnexoID',
    'link de referencia'
    ]

respostas_dano_ambiental_200_228.to_excel("respostas_dano_ambiental_200_228.xlsx")

In [ ]:
resposta_danos_ambientais_totalamostra = pd.concat([
    respostas_danoambiental_50,
    respostas_danoambiental_51_100,
    respostas_dano_ambiental_100_120,
    respostas_dano_ambiental_120_130,
    respostas_dano_ambiental_130_140,
    respostas_dano_ambiental_140_150,
    respostas_dano_ambiental_150_170,
    respostas_dano_ambiental_170_200,
    respostas_dano_ambiental_200_228
], ignore_index=True) 

In [ ]:
resposta_danos_ambientais_totalamostra.loc[resposta_danos_ambientais_totalamostra['processo_ambiental'] == 'True'].shape

(84, 5)

In [212]:
resposta_danos_ambientais_totalamostra.shape

(196, 5)